In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [18]:
train_df =pd.read_csv("train.csv")
test_df =pd.read_csv("test.csv")

In [ ]:
print(train_df.info())
print(train_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  Fals

In [ ]:
train_df= train_df.ffill().infer_objects(copy=False)
test_df= test_df.ffill().infer_objects(copy=False)

C:\Users\HP\AppData\Local\Temp\ipykernel_11372\1611568883.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df = train_df.ffill().infer_objects(copy=False)
C:\Users\HP\AppData\Local\Temp\ipykernel_11372\1611568883.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df = test_df.ffill().infer_objects(copy=False)


In [ ]:
if 'Cabin' in train_df.columns:
    train_df['Cabin'] =train_df['Cabin'].astype(str)
if 'Cabin' in test_df.columns:
    test_df['Cabin'] = test_df['Cabin'].astype(str)


In [ ]:
if isinstance(test_df, pd.DataFrame) and 'Cabin' in test_df.columns:
    test_df['CabinDeck'] =test_df['Cabin'].astype(str).str.split('/').str[0]
if isinstance(train_df, pd.DataFrame) and 'Cabin' in train_df.columns:
    train_df['CabinDeck'] =train_df['Cabin'].astype(str).str.split('/').str[0]

In [ ]:
categorical_cols=['HomePlanet','Destination','CabinDeck']
encoder =LabelEncoder()
for col in categorical_cols:
    if col in train_df.columns:
        train_df[col] =encoder.fit_transform(train_df[col].astype(str))
    if col in test_df.columns:
        test_df[col] =encoder.transform(test_df[col].astype(str))

In [ ]:
columns_to_drop=['Name','Cabin','PassengerId']
train_df.drop([col for col in columns_to_drop if col in train_df.columns],axis=1,inplace=True)
if 'PassengerId' in test_df.columns:
    test_passenger_ids = test_df['PassengerId']
test_df.drop([col for col in columns_to_drop if col in test_df.columns], axis=1,inplace=True)

In [ ]:
X = train_df.drop(columns=['Transported'])
y =train_df['Transported'].astype(int)
X_train, X_val, y_train, y_val =train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
scaler =StandardScaler()
X_train =scaler.fit_transform(X_train)
X_val =scaler.transform(X_val)
test_df =scaler.transform(test_df)

In [ ]:
model=RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred=model.predict(X_val)
print("aaccuracy:",accuracy_score(y_val,y_pred))
print(classification_report(y_val,y_pred))


aaccuracy: 0.7717078780908568
              precision    recall  f1-score   support

           0       0.77      0.76      0.77       861
           1       0.77      0.78      0.78       878

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739



In [ ]:
test_predictions=model.predict(test_df)

In [32]:
submission =pd.DataFrame({'PassengerId':test_passenger_ids,'Passengers Transported':test_predictions})
submission.to_csv("submission.csv",index=False)
print("submission file saved")

submission file saved
